In [ ]:
# default_exp models.mWDN

# multilevel Wavelet Decomposition Network (mWDN)

> This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com based on:

* Wang, J., Wang, Z., Li, J., & Wu, J. (2018, July). Multilevel wavelet decomposition network for interpretable time series analysis. In Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining (pp. 2437-2446).
* No official implementation found

In [ ]:
#export
from tsai.imports import *
from tsai.models.layers import *
from tsai.models.InceptionTime import *
from tsai.models.utils import create_model

In [ ]:
#export
import pywt

In [ ]:
#export
# This is an unofficial PyTorch implementation by Ignacio Oguiza - oguiza@gmail.com based on:

# Wang, J., Wang, Z., Li, J., & Wu, J. (2018, July). Multilevel wavelet decomposition network for interpretable time series analysis. In Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining (pp. 2437-2446).
# No official implementation found


class WaveBlock(Module):
    def __init__(self, c_in, c_out, seq_len, wavelet=None):
        if wavelet is None:
            self.h_filter = [-0.2304,0.7148,-0.6309,-0.028,0.187,0.0308,-0.0329,-0.0106]
            self.l_filter = [-0.0106,0.0329,0.0308,-0.187,-0.028,0.6309,0.7148,0.2304]
        else:
            w = pywt.Wavelet(wavelet)
            self.h_filter = w.dec_hi
            self.l_filter = w.dec_lo

        self.mWDN_H = nn.Linear(seq_len,seq_len)
        self.mWDN_L = nn.Linear(seq_len,seq_len)
        self.mWDN_H.weight = nn.Parameter(self.create_W(seq_len,False))
        self.mWDN_L.weight = nn.Parameter(self.create_W(seq_len,True))
        self.sigmoid = nn.Sigmoid()
        self.pool = nn.AvgPool1d(2)

    def forward(self,x):
        hp_1 = self.sigmoid(self.mWDN_H(x))
        lp_1 = self.sigmoid(self.mWDN_L(x))
        hp_out = self.pool(hp_1)
        lp_out = self.pool(lp_1)
        all_out = torch.cat((hp_out, lp_out), dim=-1)
        return lp_out, all_out
    
    def create_W(self, P, is_l, is_comp=False):
        if is_l: filter_list = self.l_filter
        else: filter_list = self.h_filter
        list_len = len(filter_list)
        max_epsilon = np.min(np.abs(filter_list))
        if is_comp: weight_np = np.zeros((P, P))
        else: weight_np = np.random.randn(P, P) * 0.1 * max_epsilon
        for i in range(0, P):
            filter_index = 0
            for j in range(i, P):
                if filter_index < len(filter_list):
                    weight_np[i][j] = filter_list[filter_index]
                    filter_index += 1
        return tensor(weight_np)
    
class mWDN(Module):
    def __init__(self, c_in, c_out, seq_len, levels=3, wavelet=None, arch=InceptionTime, arch_kwargs={}):
        self.levels=levels
        self.blocks = nn.ModuleList()
        for i in range(levels): self.blocks.append(WaveBlock(c_in, c_out, seq_len // 2 ** i, wavelet=wavelet))
        self.classifier = create_model(arch, c_in, c_out, seq_len=seq_len, **arch_kwargs)
        
    def forward(self,x):
        for i in range(self.levels):
            x, out_ =  self.blocks[i](x)
            if i == 0: out = out_ if i == 0 else torch.cat((out, out_), dim=-1)
        out = self.classifier(out)
        return out

In [ ]:
bs = 16
c_in = 3
seq_len = 12
c_out = 2
xb = torch.rand(bs, c_in, seq_len)
m = mWDN(c_in, c_out, seq_len)
test_eq(mWDN(c_in, c_out, seq_len)(xb).shape, [bs, c_out])

In [ ]:
m

mWDN(
  (blocks): ModuleList(
    (0): WaveBlock(
      (mWDN_H): Linear(in_features=12, out_features=12, bias=True)
      (mWDN_L): Linear(in_features=12, out_features=12, bias=True)
      (sigmoid): Sigmoid()
      (pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
    (1): WaveBlock(
      (mWDN_H): Linear(in_features=6, out_features=6, bias=True)
      (mWDN_L): Linear(in_features=6, out_features=6, bias=True)
      (sigmoid): Sigmoid()
      (pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
    (2): WaveBlock(
      (mWDN_H): Linear(in_features=3, out_features=3, bias=True)
      (mWDN_L): Linear(in_features=3, out_features=3, bias=True)
      (sigmoid): Sigmoid()
      (pool): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
    )
  )
  (classifier): InceptionTime(
    (inceptionblock): InceptionBlock(
      (inception): ModuleList(
        (0): InceptionModule(
          (bottleneck): Conv1d(3, 32, kernel_size=(1,), stride=(1,), bias=Fa

In [ ]:
#hide
out = create_scripts()
beep(out)

<IPython.core.display.Javascript object>

Converted 000_utils.ipynb.
Converted 000b_data.validation.ipynb.
Converted 001_data.external.ipynb.
Converted 002_data.core.ipynb.
Converted 003_data.transforms.ipynb.
Converted 005_data.tabular.ipynb.
Converted 007_metrics.ipynb.
Converted 008_learner.ipynb.
Converted 009_optimizer.ipynb.
Converted 010_rocket_functions.ipynb.
Converted 100_layers.ipynb.
Converted 100b_models.utils.ipynb.
Converted 101_ResNet.ipynb.
Converted 101b_ResNetPlus.ipynb.
Converted 102_InceptionTime.ipynb.
Converted 102b_InceptionTimePlus.ipynb.
Converted 103_FCN.ipynb.
Converted 103b_FCNPlus.ipynb.
Converted 104_ResCNN.ipynb.
Converted 105_RNN.ipynb.
Converted 106_XceptionTime.ipynb.
Converted 106b_XceptionTimePlus.ipynb.
Converted 107_RNN_FCN.ipynb.
Converted 108_Transformer.ipynb.
Converted 109_OmniScaleCNN.ipynb.
Converted 110_mWDN.ipynb.
Converted index.ipynb.


Checking folder: /Users/nacho/Documents/Machine_Learning/Jupyter_Notebooks/timeseries/tsai
Correct conversion! 😃
Total elapsed time 37 s
Wed, 13